In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import math
import numpy as np
import torch.nn.functional as F


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
English_file = "/kaggle/input/samanantar/final_data/en-hi/train.en"
Hindi_file = "/kaggle/input/samanantar/final_data/en-hi/train.hi"

In [5]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

In [6]:
# Hindi letters and symbols
hindi_letters = [
    'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऌ', 'ए', 'ऐ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न',
    'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'ॅ', 'े', 'ै', 'ो', 'ौ', 'ं', 'ः', 'ँ', '्', 'ऽ', 'ॐ',
    '॒', '॑', '़', '॓', '॔'
]

# Hindi numerals
hindi_numerals = [chr(code) for code in range(0x0966, 0x096F + 1)]
print(hindi_numerals)

# Common symbols and punctuation used in Hindi text
other_symbols = [
    ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    ':', ';', '<', '=', '>', '?'
]


Hindi_vocab = [START_TOKEN] + hindi_letters + hindi_numerals + other_symbols + [PADDING_TOKEN, END_TOKEN]

# Example usage
print("Hindi Vocabulary:")
print(Hindi_vocab)
print(len(Hindi_vocab))
print(len(set(Hindi_vocab)))


['०', '१', '२', '३', '४', '५', '६', '७', '८', '९']
Hindi Vocabulary:
['<START>', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऌ', 'ए', 'ऐ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'ॅ', 'े', 'ै', 'ो', 'ौ', 'ं', 'ः', 'ँ', '्', 'ऽ', 'ॐ', '॒', '॑', '़', '॓', '॔', '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '<PADDING>', '<END>']
114
114


In [7]:



English_vocab = [
    START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    ':', '<', '=', '>', '?', '@',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
    'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
    'Y', 'Z',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
    'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
    'y', 'z',
    PADDING_TOKEN, END_TOKEN
] 
 
# Print the length of English vocabulary
print("English vocabulary:", (English_vocab))


English vocabulary: ['<START>', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '<PADDING>', '<END>']


In [8]:
index_to_hindi = {k:v for k,v in enumerate(Hindi_vocab)}
hindi_to_index = {v:k for k,v in enumerate(Hindi_vocab)}
index_to_english = {k:v for k,v in enumerate(English_vocab)}
english_to_index = {v:k for k,v in enumerate(English_vocab)}

In [9]:
with open(Hindi_file, 'r') as file:
    hindi_sentences = file.readlines()
with open(English_file, 'r' ) as file:
    english_sentences = file.readlines()

In [10]:
print(len(hindi_sentences))
print(len(english_sentences))

8568307
8568307


In [11]:
for i,j  in zip(hindi_sentences[110:120],english_sentences[110:120]):
    print(j,i)


The Vice President of India, Shri M Venkaiah Naidu today urged the youth to take a firm resolve to shun negativism and join the forces of growth.
 उपराष्ट्रपति, श्री एम वेंकैया नायडू ने छात्रों से नकारात्मकता छोड़कर रचनात्‍मक दृष्टि विकसित करने का आग्रह किया है।

He is at present lodged in a prison in London.
 ऐसे में उसे अभी लंदन की जेल में दिन गुजरने होंगे.

Considering that crop segment constitutes a dominant component of the GVA computation, its performance is very critical.
 यह ध्यान में रखते हुए कि फसल खंड जीवीए कंप्यूटेशन का एक प्रमुख घटक हैं, इसका प्रदर्शन बहुत महत्वपूर्ण है।

In both conditions your doctor can prescribe you some analgesics or other medicines, or he can refer you to another specialist or surgeon.
 इन दोनों अवस्थाओं में आराम पहुंचाने के लिये आपका डाक्टर आप को दर्द नाशक अन्य दवाईयों का नुस्खा लिख कर दे सकता है या सर्जन या अन्य विशेषग्य से आप की सहायता करने को कह सकता है.

However, Prasad, who is also the Information Technology Minister, said it would not be wise 

In [12]:
total_sentences = 600000
english_sntcs = english_sentences[:total_sentences]
hindi_sntcs = hindi_sentences[:total_sentences]
english_sntcs = [sentence.rstrip('\n') for sentence in english_sntcs]
hindi_sntcs = [sentence.rstrip('\n') for sentence in hindi_sntcs]


In [13]:
%%time
max_seq_len = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to add <START> and <END> tokens

valid_sntcs_idx = []
for index in range(len(hindi_sntcs)):
    hin_sntc , eng_sntc = hindi_sntcs[index], english_sntcs[index]
    if is_valid_length(hin_sntc, max_seq_len) \
        and is_valid_length(eng_sntc, max_seq_len) \
        and is_valid_tokens(hin_sntc, Hindi_vocab)\
        and is_valid_tokens(eng_sntc, English_vocab):
        
        valid_sntcs_idx.append(index)
        
print(len(valid_sntcs_idx))


154603


In [14]:
english_sntcs = [english_sntcs[i] for i in valid_sntcs_idx]
hindi_sntcs = [hindi_sntcs[i] for i in valid_sntcs_idx]

In [15]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    
    def __init__(self, english_sntcs, hindi_sntcs):
        self.english_sntcs = english_sntcs
        self.hindi_sntcs = hindi_sntcs
        
    def __len__(self):
        return len(self.english_sntcs)
    
    def __getitem__(self,idx):
        return self.english_sntcs[idx], self.hindi_sntcs[idx]


In [16]:
dataset = TextDataset(english_sntcs, hindi_sntcs)
len(dataset)

154603

In [17]:
batch_size = 32

train_loader = DataLoader(dataset, batch_size)
len(train_loader)

4832

In [32]:
import torch

d_model = 512
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 2
max_sequence_length = max_seq_len = 200
hin_vocab_size = len(Hindi_vocab)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_seq_len,
                          hin_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)


In [33]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=0.0001)

In [34]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
        eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
        eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
        kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [35]:
import torch
import numpy as np

NEG_INFTY = -1e9

def create_masks1(eng_batch, hin_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_seq_len, max_seq_len] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_seq_len, max_seq_len] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_seq_len, max_seq_len] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_seq_len, max_seq_len] , False)
    
    for idx in range(num_sentences):
        eng_sentence_length, hin_sentence_length = len(eng_batch[idx]), len(hin_batch[idx])
#         print(eng_sentence_length,hin_sentence_length)
        eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_seq_len)
        hin_chars_to_padding_mask = np.arange(hin_sentence_length + 1, max_seq_len)
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, :, hin_chars_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, hin_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, hin_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    print(f"encoder_self_attention_mask {encoder_self_attention_mask.size()}: {encoder_self_attention_mask[0, :50, :50]}")
    print(f"decoder_self_attention_mask {decoder_self_attention_mask.size()}: {decoder_self_attention_mask[0, :50, :50]}")
    print(f"decoder_cross_attention_mask {decoder_cross_attention_mask.size()}: {decoder_cross_attention_mask[0, :50, :50]}")
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask
        
# Corrected function call with provided sentences
hin_sentence = ("आप कैसे हैं",)
eng_sentence = ("should we go to the mall?",)
encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks1(eng_sentence, hin_sentence)

encoder_self_attention_mask torch.Size([1, 200, 200]): tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
         -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
         -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
         -1.0000e+09, -1.0000e+09],
        ...,
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
         -1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
         -1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
         -1.0000e+09, -1.0000e+09]])
decoder_self_attention_mask torch.Size([1, 200, 200]): tensor([[ 0.0000e+00, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
         -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00, -1.0000e+09,  ..., -1.0000e+09,
         -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...

In [39]:
%%time
transformer.train()
print(device)
transformer.to(device)
total_loss = 0
num_epochs = 10
print(train_loader)
for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    print(len(train_loader))
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, hin_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hin_batch)
#         print(encoder_self_attention_mask.shape,decoder_self_attention_mask.shape, decoder_cross_attention_mask.shape)
#         print(encoder_self_attention_mask[0][0],encoder_self_attention_mask[1][0],encoder_self_attention_mask[0][2])
        optim.zero_grad()
        hin_predictions = transformer(eng_batch,
                                     hin_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hin_batch, start_token=False, end_token=True)
        loss = criterian(
            hin_predictions.view(-1, hin_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == hindi_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 1000 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hin_batch[0]}")
#             print("Mask",encoder_self_attention_mask[0,:50,:50])
            hin_sentence_predicted = torch.argmax(hin_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in hin_sentence_predicted:
                if idx == hindi_to_index[END_TOKEN]:
                    break
                predicted_sentence += index_to_hindi[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")


            transformer.eval()
            hin_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
#             print(encoder_self_attention_mask.shape,decoder_self_attention_mask.shape, decoder_cross_attention_mask.shape)
#             print(encoder_self_attention_mask[0][0],encoder_self_attention_mask[0][1],encoder_self_attention_mask[0][2])
               
            for word_counter in range(max_seq_len):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hin_sentence)
                predictions = transformer(eng_sentence,
                                          hin_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_hindi[next_token_index]
                hin_sentence = (hin_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                    break
            
            print(f"Evaluation translation (should we go to the mall?) : {hin_sentence}")
            print("-------------------------------------------")

cuda
Epoch 0
4832
Iteration 0 : 4.214349269866943
English: In reply, Pakistan got off to a solid start.
Hindi Translation: जिसके जवाब में पाक ने अच्छी शुरुआत की थी.
Hindi Prediction: कंककाकाककककूकककरकककााकाककककककककरकककककााकककहंाूंहंकहररारााकाहंहहहहहाह्हहहकानहहह कह करर ककरंाकारहरर्ाकंाहककर ंेक  कह ह्क रकरहकककि हहकाहककह काहहहहककहहहरहकाहहाहाााााहरारांहांाहहाकरकहाहााााकाूााााकहककककरका
Evaluation translation (should we go to the mall?) : ('ाााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााााा',)
-------------------------------------------
Iteration 1000 : 2.6333467960357666
English: """The BJP is only doing politics over cow,"" he claimed."
Hindi Translation: उन्होंने कहा कि बीजेपी यूज एंड थ्रो की राजनीति करती है.
Hindi Prediction: इनेहों   केा के का ा ् कारीक  ़कीर ंके केरा   यहे ा कै.
Evaluation translation (should we go to the mall?) : ('इस मे

In [40]:
print(hindi_to_index)

{'<START>': 0, 'अ': 1, 'आ': 2, 'इ': 3, 'ई': 4, 'उ': 5, 'ऊ': 6, 'ऋ': 7, 'ऌ': 8, 'ए': 9, 'ऐ': 10, 'ओ': 11, 'औ': 12, 'क': 13, 'ख': 14, 'ग': 15, 'घ': 16, 'ङ': 17, 'च': 18, 'छ': 19, 'ज': 20, 'झ': 21, 'ञ': 22, 'ट': 23, 'ठ': 24, 'ड': 25, 'ढ': 26, 'ण': 27, 'त': 28, 'थ': 29, 'द': 30, 'ध': 31, 'न': 32, 'प': 33, 'फ': 34, 'ब': 35, 'भ': 36, 'म': 37, 'य': 38, 'र': 39, 'ल': 40, 'ळ': 41, 'व': 42, 'श': 43, 'ष': 44, 'स': 45, 'ह': 46, 'ा': 47, 'ि': 48, 'ी': 49, 'ु': 50, 'ू': 51, 'ृ': 52, 'ॄ': 53, 'ॅ': 54, 'े': 55, 'ै': 56, 'ो': 57, 'ौ': 58, 'ं': 59, 'ः': 60, 'ँ': 61, '्': 62, 'ऽ': 63, 'ॐ': 64, '॒': 65, '॑': 66, '़': 67, '॓': 68, '॔': 69, '०': 70, '१': 71, '२': 72, '३': 73, '४': 74, '५': 75, '६': 76, '७': 77, '८': 78, '९': 79, ' ': 80, '!': 81, '"': 82, '#': 83, '$': 84, '%': 85, '&': 86, "'": 87, '(': 88, ')': 89, '*': 90, '+': 91, ',': 92, '-': 93, '.': 94, '/': 95, '0': 96, '1': 97, '2': 98, '3': 99, '4': 100, '5': 101, '6': 102, '7': 103, '8': 104, '9': 105, ':': 106, ';': 107, '<': 108, '=': 109, '>'

In [41]:
transformer.eval()
def translate(eng_sentence):
    eng_sentence = (eng_sentence,)
    kn_sentence = ("",)
    for word_counter in range(max_sequence_length):
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
        predictions = transformer(eng_sentence,
                                  kn_sentence,
                                  encoder_self_attention_mask.to(device), 
                                  decoder_self_attention_mask.to(device), 
                                  decoder_cross_attention_mask.to(device),
                                  enc_start_token=False,
                                  enc_end_token=False,
                                  dec_start_token=True,
                                  dec_end_token=False)
        next_token_prob_distribution = predictions[0][word_counter]
        next_token_index = torch.argmax(next_token_prob_distribution).item()
        next_token = index_to_hindi[next_token_index]
        kn_sentence = (kn_sentence[0] + next_token, )
        if next_token == END_TOKEN:
            break
    return kn_sentence[0]

In [42]:
translation = translate("Don't do that")
print(translation)

न करें न करें<END>


In [43]:
translation = translate("where are you?")
print(translation)

कहां कहां है?<END>


In [44]:
translation = translate("who are you?")
print(translation)

किसे किसके पास किसके ?<END>


In [55]:
translation = translate("I am fine")
print(translation)

मैं मैं कर रहा हूँ<END>


In [61]:
import sentencepiece as spm

# Example Hindi text data
train_data = """
मैं मशीन लर्निंग सीख रहा हूं|
यह मेरा दूसरा उदाहरण है|
"""

# Save the training data to a file
with open('hindi_text.txt', 'w', encoding='utf-8') as f:
    f.write(train_data)

# Train SentencePiece model with a target vocabulary size
vocab_size = 50  # You can adjust this size as needed

spm.SentencePieceTrainer.train(input='hindi_text.txt', model_prefix='hindi', vocab_size=vocab_size, character_coverage=1.0, model_type='bpe')

# Load the trained model
sp = spm.SentencePieceProcessor()
sp.load('hindi.model')

# Test the tokenizer
test_sentence = "यह सिर्फ एक उदाहरण है|"
tokens = sp.encode_as_pieces(test_sentence)
token_ids = sp.encode_as_ids(test_sentence)

print("Tokens:", tokens)
print("Token IDs:", token_ids)


Tokens: ['▁यह', '▁', 'स', 'ि', 'र्', 'फ', '▁', 'एक', '▁उद', 'ाह', 'रण', '▁ह', 'ै', '|']
Token IDs: [26, 27, 36, 47, 13, 0, 27, 0, 25, 16, 11, 5, 39, 33]


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: hindi_text.txt
  input_format: 
  model_prefix: hindi
  model_type: BPE
  vocab_size: 50
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_priv

In [18]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled = scaled.permute(1, 0, 2, 3) + mask
        scaled = scaled.permute(1, 0, 2, 3)
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [19]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = (torch.arange(self.max_sequence_length)
                          .reshape(self.max_sequence_length, 1))
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        return PE

In [20]:
class SentenceEmbedding(nn.Module):
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN
    
    def batch_tokenize(self, batch, start_token, end_token):
        def tokenize(sentence, start_token, end_token):
            sentence_word_indicies = [self.language_to_index[token] for token in list(sentence)]
            if start_token:
                sentence_word_indicies.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indicies.append(self.language_to_index[self.END_TOKEN])
            for _ in range(len(sentence_word_indicies), self.max_sequence_length):
                sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
            return torch.tensor(sentence_word_indicies)

        tokenized = []
        for sentence_num in range(len(batch)):
            tokenized.append( tokenize(batch[sentence_num], start_token, end_token) )
        tokenized = torch.stack(tokenized)
        return tokenized.to(device)
    
    def forward(self, x, start_token, end_token): # sentence
        x = self.batch_tokenize(x, start_token, end_token)
        x = self.embedding(x)
        pos = self.position_encoder().to(device)
        x = self.dropout(x + pos)
        return x

In [21]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model , 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, mask):
        batch_size, sequence_length, d_model = x.size()
        qkv = self.qkv_layer(x)
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3)
        q, k, v = qkv.chunk(3, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, self.num_heads * self.head_dim)
        out = self.linear_layer(values)
        return out

In [22]:
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std
        out = self.gamma * y + self.beta
        return out

In [23]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [24]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, self_attention_mask):
        residual_x = x.clone()
        x = self.attention(x, mask=self_attention_mask)
        x = self.dropout1(x)
        x = self.norm1(x + residual_x)
        residual_x = x.clone()
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x + residual_x)
        return x # 30 x 200 x 512

In [25]:
class SequentialEncoder(nn.Sequential):
    def forward(self, *inputs):
        x, self_attention_mask  = inputs
        for module in self._modules.values():
            x = module(x, self_attention_mask)
        return x

In [26]:
class Encoder(nn.Module):
    def __init__(self, 
                 d_model, 
                 ffn_hidden, 
                 num_heads, 
                 drop_prob, 
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN, 
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialEncoder(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                      for _ in range(num_layers)])

    def forward(self, x, self_attention_mask, start_token, end_token):
        x = self.sentence_embedding(x, start_token, end_token)
        x = self.layers(x, self_attention_mask)
        return x

In [27]:
class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model , 2 * d_model)
        self.q_layer = nn.Linear(d_model , d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, y, mask):
        batch_size, sequence_length, d_model = x.size() # in practice, this is the same for both languages...so we can technically combine with normal attention
        kv = self.kv_layer(x)
        q = self.q_layer(y)
        kv = kv.reshape(batch_size, sequence_length, self.num_heads, 2 * self.head_dim)
        q = q.reshape(batch_size, sequence_length, self.num_heads, self.head_dim)
        kv = kv.permute(0, 2, 1, 3)
        q = q.permute(0, 2, 1, 3)
        k, v = kv.chunk(2, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask) # We don't need the mask for cross attention, removing in outer function!
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, d_model)
        out = self.linear_layer(values)
        return out

In [28]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.layer_norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self, x, y, self_attention_mask, cross_attention_mask):
        _y = y.clone()
        y = self.self_attention(y, mask=self_attention_mask)
        y = self.dropout1(y)
        y = self.layer_norm1(y + _y)

        _y = y.clone()
        y = self.encoder_decoder_attention(x, y, mask=cross_attention_mask)
        y = self.dropout2(y)
        y = self.layer_norm2(y + _y)

        _y = y.clone()
        y = self.ffn(y)
        y = self.dropout3(y)
        y = self.layer_norm3(y + _y)
        return y

In [29]:
class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x, y, self_attention_mask, cross_attention_mask = inputs
        for module in self._modules.values():
            y = module(x, y, self_attention_mask, cross_attention_mask)
        return y

In [30]:
class Decoder(nn.Module):
    def __init__(self, 
                 d_model, 
                 ffn_hidden, 
                 num_heads, 
                 drop_prob, 
                 num_layers,
                 max_seq_len,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN, 
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_seq_len, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

    def forward(self, x, y, self_attention_mask, cross_attention_mask, start_token, end_token):
        y = self.sentence_embedding(y, start_token, end_token)
        y = self.layers(x, y, self_attention_mask, cross_attention_mask)
        return y

In [31]:
class Transformer(nn.Module):
    def __init__(self, 
                d_model, 
                ffn_hidden, 
                num_heads, 
                drop_prob, 
                num_layers,
                max_seq_len, 
                hin_vocab_size,
                english_to_index,
                hindi_to_index,
                START_TOKEN, 
                END_TOKEN, 
                PADDING_TOKEN
                ):
        super().__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_seq_len, english_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_seq_len, hindi_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.linear = nn.Linear(d_model, hin_vocab_size)

    def forward(self, 
                x, 
                y, 
                encoder_self_attention_mask=None, 
                decoder_self_attention_mask=None, 
                decoder_cross_attention_mask=None,
                enc_start_token=False,
                enc_end_token=False,
                dec_start_token=False, # We should make this true
                dec_end_token=False): # x, y are batch of sentences
        x = self.encoder(x, encoder_self_attention_mask, start_token=enc_start_token, end_token=enc_end_token)
        out = self.decoder(x, y, decoder_self_attention_mask, decoder_cross_attention_mask, start_token=dec_start_token, end_token=dec_end_token)
        out = self.linear(out)
        return out